The idea is to build a model to take as input a sequence of characters representing a name, and find the associated country.

Inspired by: https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial

# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import random
import matplotlib.pyplot as plt

import unicodedata
import string

# Data

Reading the data from the files.

In [2]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


In [3]:
categories = []
category_lines = {}

for (dirpath, dirnames, filenames) in os.walk('data/names'):
    for (i, filename) in enumerate([dirpath + os.sep + f for f in filenames]):
        category = os.path.splitext(os.path.basename(filename))[0]
        categories.append(category)
        lines = open(filename, encoding='utf-8').read().lower().strip().split('\n')
        category_lines[category] = [unicode_to_ascii(line) for line in lines]

In [4]:
def str_to_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, n_letters)
    for (i, c) in enumerate(name):
        tensor[i] = F.one_hot(torch.Tensor([all_letters.find(c)]).long(), num_classes=n_letters)
        
    return tensor

In [5]:
category_lines

{'Arabic': ['khoury',
  'nahas',
  'daher',
  'gerges',
  'nazari',
  'maalouf',
  'gerges',
  'naifeh',
  'guirguis',
  'baba',
  'sabbagh',
  'attia',
  'tahan',
  'haddad',
  'aswad',
  'najjar',
  'dagher',
  'maloof',
  'isa',
  'asghar',
  'nader',
  'gaber',
  'abboud',
  'maalouf',
  'zogby',
  'srour',
  'bahar',
  'mustafa',
  'hanania',
  'daher',
  'tuma',
  'nahas',
  'saliba',
  'shamoon',
  'handal',
  'baba',
  'amari',
  'bahar',
  'atiyeh',
  'said',
  'khouri',
  'tahan',
  'baba',
  'mustafa',
  'guirguis',
  'sleiman',
  'seif',
  'dagher',
  'bahar',
  'gaber',
  'harb',
  'seif',
  'asker',
  'nader',
  'antar',
  'awad',
  'srour',
  'shadid',
  'hajjar',
  'hanania',
  'kalb',
  'shadid',
  'bazzi',
  'mustafa',
  'masih',
  'ghanem',
  'haddad',
  'isa',
  'antoun',
  'sarraf',
  'sleiman',
  'dagher',
  'najjar',
  'malouf',
  'nahas',
  'naser',
  'saliba',
  'shamon',
  'malouf',
  'kalb',
  'daher',
  'maalouf',
  'wasem',
  'kanaan',
  'naifeh',
  'boutro

# Neural network module

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden
        
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [7]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, len(categories))

# Training

In [8]:
criterion = nn.NLLLoss()
learning_rate = 0.005
n_iters = 100000

In [9]:
def get_training_example():
    category = random.choice(list(categories))
    line = random.choice(category_lines[category])
    category_tensor = torch.tensor([categories.index(category)], dtype=torch.long)
    line_tensor = str_to_tensor(line)
    return category_tensor, line_tensor

In [10]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

def train(category_tensor, line_tensor):
    hidden = rnn.init_hidden()
    
    rnn.zero_grad()
    
    for i in range (line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
    loss = criterion(output, category_tensor)
    loss.backward()
    
    # optimizer.step()
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return output, loss.item()

In [11]:
all_losses = []
current_loss = 0
for iter in range(1, n_iters + 1):
    category_tensor, line_tensor = get_training_example()
    output, loss = train(category_tensor, line_tensor)
    if iter % 100 == 0:
        print(iter, ":", torch.Tensor(all_losses[-100:]).mean())
        all_losses.append(loss)
    current_loss += loss

100 : tensor(nan)
200 : tensor(2.8408)
300 : tensor(2.9118)
400 : tensor(2.8616)
500 : tensor(2.8725)
600 : tensor(2.8204)
700 : tensor(2.8514)
800 : tensor(2.8406)
900 : tensor(2.8748)
1000 : tensor(2.8929)
1100 : tensor(2.8863)
1200 : tensor(2.8635)
1300 : tensor(2.8797)
1400 : tensor(2.8773)
1500 : tensor(2.8825)
1600 : tensor(2.8719)
1700 : tensor(2.8784)
1800 : tensor(2.8734)
1900 : tensor(2.8568)
2000 : tensor(2.8666)
2100 : tensor(2.8593)
2200 : tensor(2.8654)
2300 : tensor(2.8281)
2400 : tensor(2.8234)
2500 : tensor(2.8237)
2600 : tensor(2.8244)
2700 : tensor(2.8096)
2800 : tensor(2.8158)
2900 : tensor(2.8261)
3000 : tensor(2.8290)
3100 : tensor(2.8295)
3200 : tensor(2.8435)
3300 : tensor(2.8238)
3400 : tensor(2.8139)
3500 : tensor(2.8009)
3600 : tensor(2.7778)
3700 : tensor(2.7903)
3800 : tensor(2.8184)
3900 : tensor(2.8378)
4000 : tensor(2.8289)
4100 : tensor(2.8231)
4200 : tensor(2.8021)
4300 : tensor(2.8072)
4400 : tensor(2.8062)
4500 : tensor(2.8288)
4600 : tensor(2.8057)


In [12]:
# plt.figure()
# plt.plot(all_losses)
# plt.show()

# Evaluation of results

In [23]:
line_tensor = str_to_tensor("lombard")

hidden = rnn.init_hidden()

for i in range (line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)
        
print(categories[output.argmax().item()])

French
